In [ ]:

!git clone https://github.com/MaastrichtU-IDS/LMKBC-2023.git

In [ ]:
!git reset --hard
!git pull 

In [ ]:
!git pull

In [ ]:
import os
os.chdir('LMKBC-2023/')

In [ ]:
!pip install accelerate -U
!pip install transformers

In [ ]:
!python src/run.py --mode "train_fm"



In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
!git add . 
!git commit -m 'from server' 
!git push


In [ ]:
#!model_load_dir = '/workspace/persistent/LMKBC-2023/bin/pretrain_fill_mask/bert-large-cased/best_ckpt'
!model_load_dir ='/workspace/persistent/LMKBC-2023/bin/train_full/fill_mask/bert-large-cased/best_ckpt'

!python  /workspace/persistent/LMKBC-2023/src/fm_model.py  --test_fn /workspace/persistent/LMKBC-2023/data/val.jsonl --template_fn res/prompts0.csv   --output_fn /workspace/persistent/LMKBC-2023/output/filled-mask-valid.jsonl --train_fn /workspace/persistent/LMKBC-2023/data/train.jsonl          --train_batch_size 256 --gpu 0  --top_k 30 --threshold 0.1  --dev_fn  /workspace/persistent/LMKBC-2023/data/train_tiny.jsonl         --mode "tr ain test"  --train_epoch 50 --learning_rate 5e-5         --model_load_dir   $model_load_dir         --model_save_dir  /workspace/persistent/LMKBC-2023/bin/train_full/fill_mask/bert-large-cased        --model_best_dir  /workspace/persistent/LMKBC-2023/bin/train_full/fill_mask/bert-large-cased/best_ckpt 

In [ ]:
   python {config.SRC_DIR}/fm_model.py  --test_fn {test_fn} --template_fn res/prompts0.csv  --output_fn {OUTPUT_FILE} --train_fn {config.DATA_DIR}/train.jsonl --train_batch_size 64 --gpu {args.gpu}  --top_k 30 --threshold 0.1  --dev_fn  {config.DATA_DIR}/train_tiny.jsonl --mode "train test" --train_epoch 50 --learning_rate 5e-5 --model_load_dir {last_best_dir} --model_save_dir {fm_model_save_dir} --model_best_dir  {fm_model_best_dir}

In [ ]:
!nvidia-smi

In [ ]:
!cd /workspace/persistent/LMKBC-2023/
import os

command = '''
nohup python src/run.py --label 'mask-random' --task pfm,fm --load_dir bert-base-cased --epoch 20,50 --mode 'random,' &
'''
os.system(command)

command = '''
nohup python src/run.py --label 'mask-single' --task pfm,fm --load_dir bert-base-cased --epoch 20,50 --mode 'single,' &
'''
os.system(command)

command = '''
nohup python src/run.py --label 'mask-fold' --task pfm,fm --load_dir bert-base-cased --epoch 20,50 --mode 'fold,' &
'''
os.system(command)

In [ ]:
import src.config 

label = 'mask-random'
pfm_model_save_dir =f"bin/{label}/pfm"
pfm_model_best_dir = pfm_model_save_dir + "/best_ckpt"
input_fp = f"{config.RES_DIR}/wikidata/Country-Language-State/filter.json"
model_load_dir = config.bert_base_cased
p

cmd_pretrain_filled_mask = f"""

python src/pre_model.py   --train_fn {input_fp}  --train_batch_size 32 --gpu 0  --train_epoch 32 --learning_rate 5e-5  --mask_strategy random 
 --model_load_dir {model_load_dir} --model_save_dir {pfm_model_save_dir} --model_best_dir  {pfm_model_best_dir}

"""

os.system(cmd_pretrain_filled_mask)


OUTPUT_FILE = f'output/filled-mask-valid.jsonl'

fm_model_save_dir =f"bin/label/fm"
fm_model_best_dir = fm_model_save_dir + "/best_ckpt"
test_fn = config.VAL_FN

cmd_run_fillmask = f"""

python src/fm_model.py  --test_fn data/val.jsonl --template_fn res/prompts0.csv  --output_fn {OUTPUT_FILE} --train_fn data/train.jsonl 
  --train_batch_size 64 --gpu 0  --top_k 30 --threshold 0.1  --dev_fn  data/train_tiny.jsonl --mode "train test"   
  --train_epoch 50 --learning_rate 5e-5 --model_load_dir {pfm_model_best_dir} --model_save_dir {fm_model_save_dir} 
  --model_best_dir  {fm_model_best_dir}

"""

os.system(cmd_run_fillmask)

    

In [ ]:
!cat nohup.out 

In [ ]:
  !python /workspace/persistent/LMKBC-2023/src/fm_model.py  --test_fn /workspace/persistent/LMKBC-2023/data/val.jsonl --template_fn res/prompts0.csv  --output_fn /workspace/persistent/LMKBC-2023/output/filled-mask-valid.jsonl --train_fn /workspace/persistent/LMKBC-2023/data/train.jsonl --train_batch_size 64 --gpu 0  --top_k 30 --threshold 0.1  --dev_fn  /workspace/persistent/LMKBC-2023/data/train_tiny.jsonl --mode "train test" --train_epoch 50 --learning_rate 5e-5 --model_load_dir /workspace/persistent/LMKBC-2023/bin/default/pretrain_fill_mask/best_ckpt --model_save_dir /workspace/persistent/LMKBC-2023/bin/default/fill_mask --model_best_dir  /workspace/persistent/LMKBC-2023/bin/default/fill_mask/best_ckpt

In [60]:
!tail -n 1000  nohup.out| grep loss

{'loss': 7.3641, 'learning_rate': 2.9909404659188957e-05, 'epoch': 1.0}
{'loss': 4.3889, 'learning_rate': 2.6678170836928386e-05, 'epoch': 2.0}
{'loss': 2.5687, 'learning_rate': 2.334627552487777e-05, 'epoch': 3.0}
{'loss': 1.5024, 'learning_rate': 2.0012942191544437e-05, 'epoch': 4.0}
{'loss': 0.8552, 'learning_rate': 1.66796088582111e-05, 'epoch': 5.0}
{'loss': 0.4638, 'learning_rate': 1.3347713546160483e-05, 'epoch': 6.0}
{'loss': 0.2462, 'learning_rate': 1.001438021282715e-05, 'epoch': 7.0}
{'loss': 0.1276, 'learning_rate': 6.681046879493816e-06, 'epoch': 8.0}
{'loss': 0.0654, 'learning_rate': 3.3491515674431984e-06, 'epoch': 9.0}
{'loss': 0.0367, 'learning_rate': 1.5818234109864827e-08, 'epoch': 10.0}
{'train_runtime': 9001.6863, 'train_samples_per_second': 41.199, 'train_steps_per_second': 2.575, 'train_loss': 1.7618949900010827, 'epoch': 10.0}
{'loss': 7.5764, 'learning_rate': 3.329048843187661e-05, 'epoch': 1.0}
{'loss': 4.8136, 'learning_rate': 4.8154514756740766e-05, 'epoch':

In [70]:
import json 
import src.util
result_fp = 'res/results.jsonl'
sentence_list =  open(result_fp).readlines()
json_list = [json.loads(s) for s in sentence_list] 

result_json = json.loads(open(result_fp).readlines()[-10])
label = result_json['args']['model_save_dir']
print(label)
need_keys = ['CountryBordersCountry','CountryHasOfficialLanguage',
             'CountryHasStates',"StateBordersState",'Average']
result = result_json['metric']
#print(result.keys())
for k in need_keys:
    print(k, json.dumps(result[k],indent = 2))


/workspace/persistent/LMKBC-2023/bin/fill_mask/bert-large-cased
CountryBordersCountry {
  "p": 0.08783068783068781,
  "r": 0.6477891156462586,
  "f1": 0.15071464504439727,
  "best_threshold": 0.03,
  "best_f1": 0.26672353219972267
}
CountryHasOfficialLanguage {
  "p": 0.03540229885057473,
  "r": 0.7412587412587412,
  "f1": 0.06654891825287991,
  "best_threshold": 0.13,
  "best_f1": 0.48923076923076936
}
CountryHasStates {
  "p": 0.0014492753623188406,
  "r": 0.004693675889328063,
  "f1": 0.002204610146787967,
  "best_threshold": 0.01,
  "best_f1": 0.00380952380952381
}
StateBordersState {
  "p": 0.006356321839080459,
  "r": 0.04651298701298701,
  "f1": 0.01072487564061384,
  "best_threshold": 0.01,
  "best_f1": 0.022188375350140063
}
Average {
  "p": 0.028341648371169318,
  "r": 0.5173254574470191,
  "f1": 0.052348897664377544,
  "best_f1": 0.24164248119666945
}
